# Creating a Dataset for Retail Banking Conversations.
This notebook contains an example of how to use an existing profile file and prompt templates to generate a dataset of financial conversations using the WizardSData library.

# Install and import Libraries. 

In [1]:
#Install library
%pip install -q wizardsdata
%pip install -q pandas
%pip install -q dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
#Import class and json
import wizardsdata as wsd
import json

## Loading and studying profiles & Prompt templates. 

In [3]:
#This file contains 5 different profiles. 
file_profiles = "../../templates/financial01/profiles/financial_sample01_5.json"

In [4]:
with open(file_profiles, 'r') as file:
    data = json.load(file)

In [5]:
data

{'profiles': [{'id': 1,
   'age': 30,
   'marital_status': 'Single',
   'country': 'Spain',
   'residence_area': 'Urban',
   'profession': 'Software Developer',
   'employment_status': 'Employed',
   'financial_products': ['Savings account', 'Tech stocks'],
   'financial_goal': 'Save for house deposit',
   'investment_horizon': 'Medium-term',
   'risk_tolerance': 'Moderate',
   'financial_knowledge': 'Intermediate'},
  {'id': 2,
   'age': 45,
   'marital_status': 'Married',
   'country': 'USA',
   'residence_area': 'Suburb',
   'profession': 'Marketing Manager',
   'employment_status': 'Employed',
   'financial_products': ['401k', 'Index funds'],
   'financial_goal': 'Plan for retirement',
   'investment_horizon': 'Long-term',
   'risk_tolerance': 'Low',
   'financial_knowledge': 'Intermediate'},
  {'id': 3,
   'age': 60,
   'marital_status': 'Widowed',
   'country': 'UK',
   'residence_area': 'Urban',
   'profession': 'Retired Banker',
   'employment_status': 'Retired',
   'financial_

At the top, you can see the five profiles included in the file. These profiles have been specifically created for this example. The only mandatory field is id; the rest of the fields have been designed explicitly for this use case.

The fields are used to populate the prompt templates, which must also be created before calling the library. These templates provide the necessary instructions to the language models.

The prompt files are in j2 format, as Jinja2 is used to populate them with the profile content.

In this example, the first profile represents the client. The client is given an identity based on the demographic characteristics defined in the profile, along with their interest in the conversation.

The second profile represents the advisor. The advisor does not receive direct access to the client’s data—only the information they could infer by seeing them in person. This ensures they need to ask for additional details, leading to more realistic conversations.

In this case, both profiles have been instructed to use the [END] tag to signal the end of the conversation. This is a design decision, but it is also possible to assign the responsibility of closing the conversation to only one of the roles. In fact, this use case has been tested by allowing only the client to end the conversation, and the generated dialogues have been equally effective.

**Prompt Client**
```txt
You are a {{ profile.age }}-year-old {{ profile.marital_status | lower }} client living in a {{ profile.residence_area | lower }} area of {{ profile.country }}. 
You work as a {{ profile.profession | lower }} and have {{ profile.financial_knowledge | lower }} financial knowledge. 
You currently have {{ profile.financial_products | join(' and ') }}. 
Your main financial goal is to {{ profile.financial_goal | lower }} in the {{ profile.investment_horizon | lower }}. 
You have a {{ profile.risk_tolerance | lower }} risk tolerance and are looking for advice on how to improve your saving and investment strategy.

You are having a conversation with a financial advisor.
- Your first message should be a BRIEF, CASUAL greeting. Don't reveal all your financial details at once.
- For example, just say hi and mention ONE thing like wanting advice about saving or investments.
- Keep your first message under 15-30 words. Let the conversation develop naturally.
- In later messages, respond naturally to the advisor's questions, revealing information gradually.
- Provide ONLY your next message as the client. Do not simulate the advisor's responses.
- Start with a natural greeting if this is your first message.
- Ask relevant questions or express concerns to achieve your goal.
- Respond naturally and concisely to the advisor's previous message.
- Try to conclude the conversation in fewer than {{ max_questions }} exchanges.
- If you feel your questions are resolved, end your message with '[END]'.
```
**Financial Advisor Prompt.**
```txt
You are an expert financial advisor specializing in {{ profile.financial_goal | lower }}.

Client Context:
- The client is approximately {{ profile.age }} years old, {{ profile.marital_status | lower }}, and appears to be a {{ profile.profession | lower }} from {{ profile.country }}.
- The client's financial goal is to {{ profile.financial_goal | lower }}.

Instructions for the conversation:
- Start by greeting the client and asking relevant, natural questions to understand their financial situation, preferences, and concerns.
- Guide the conversation by asking about their current financial products, investment experience, and risk tolerance.
- Provide clear, concise, and professional advice tailored to the client's goal and profile as the information is revealed.
- Avoid using complex financial jargon unless necessary, and adapt your language to the client's knowledge level (you'll assess this through conversation).
- Focus on actionable recommendations to help the client achieve their goal.
- Keep the conversation realistic and friendly.
- End the conversation naturally once you believe the client's doubts have been resolved, or explicitly conclude by saying '[END]'
```


Both the profile file and the files containing the prompt templates must be provided in the configuration.

## Configuration. 

In [6]:
errors = wsd.set_config(
        API_KEY="YOUR-API-KEY",  # Replace with your actual API key
        template_client_prompt="../../templates/financial01/prompts/financial_client_01.j2",
        template_advisor_prompt="templates/financial01/prompts/financial_advisor_01.j2",
        file_profiles="templates/financial01/profiles/financial_sample01_5.json",
        file_output="templates/financial01/outputs/test_dataset01_1.json",
        model_client="gpt-4o-mini",
        model_advisor="gpt-4o-mini",
        # Optional parameters with custom values
        temperature_client=0.8,
        temperature_advisor=0.1, 
        max_recommended_questions=15
    )

In [7]:
errors

['template_advisor_prompt (file not found: templates/financial01/prompts/financial_advisor_01.j2)',
 'file_profiles (file not found: templates/financial01/profiles/financial_sample01_5.json)']

The configuration returns a list of errors indicating any issues that need to be corrected in the parameters. If the list is empty, the configuration is valid, and the generation process can begin.

In [8]:
api_key=None
print(api_key)

None


In [9]:
from dotenv import load_dotenv
import os
load_dotenv(dotenv_path='../../config.env')
api_key = os.environ.get("OPENAI_API_KEY")

In [12]:
errors = wsd.set_config(
        API_KEY=api_key,  # Replace with your actual API key
        template_client_prompt="../../templates/financial01/prompts/financial_client_01.j2",
        template_advisor_prompt="../../templates/financial01/prompts/financial_advisor_01.j2",
        file_profiles="../../templates/financial01/profiles/financial_sample01_5.json",
        file_output="./test_financial_dataset01_5.json",
        model_client="gpt-4o-mini",
        model_advisor="gpt-4o-mini",
        # Optional parameters with custom values
        temperature_client=0.8,
        temperature_advisor=0.1, 
        max_recommended_questions=15
    )

In [13]:
errors

[]

In this case, the list is empty, indicating that the configuration is valid.

## Generation

In [14]:
wsd.start_generation()

client: Hi there! I’m looking for some advice on saving for a house deposit.
advisor: Hello! It's great to hear that you're looking to save for a house deposit. That's an exciting goal! To get started, could you tell me a bit about your current financial situation? For example, do you have any savings set aside already, and what is your target amount for the deposit?
client: Sure! I have a savings account and some tech stocks, but I'm not sure how much I should be aiming for in total. Any tips on setting a target amount?
advisor: Absolutely! Setting a target amount for your house deposit is a crucial step. Generally, a common benchmark is to aim for 20% of the property's purchase price to avoid private mortgage insurance (PMI). However, this can vary based on your location and the type of property you're interested in.

To help you set a target, consider these questions:
1. What is your ideal location for buying a house?
2. Have you researched the average property prices in that area?


True

In [15]:
import pandas as pd

# Load the JSON file
file_path = 'test_financial_dataset01_5.json'
data = pd.read_json(file_path)

In [16]:
data[0:20]

,id_conversation,topic,sequence,rol1,rol2
0,7de9ac56-4b0f-486c-acb1-63f8df7c4f8d,Save for house deposit,0,Hi there! I’m looking for some advice on savin...,Hello! It's great to hear that you're looking ...
1,7de9ac56-4b0f-486c-acb1-63f8df7c4f8d,Save for house deposit,2,Sure! I have a savings account and some tech s...,Absolutely! Setting a target amount for your h...
2,7de9ac56-4b0f-486c-acb1-63f8df7c4f8d,Save for house deposit,4,"I'm looking at urban areas in Spain, maybe aro...",Focusing on a specific type of property is a s...
3,7de9ac56-4b0f-486c-acb1-63f8df7c4f8d,Save for house deposit,6,I’m leaning towards a one-bedroom apartment. I...,Great choice! For a one-bedroom apartment pric...
4,7de9ac56-4b0f-486c-acb1-63f8df7c4f8d,Save for house deposit,8,"I currently have about €15,000 saved up. I can...",Let’s break it down based on your current savi...
5,7de9ac56-4b0f-486c-acb1-63f8df7c4f8d,Save for house deposit,10,"Yes, I’d like to explore some options to boost...",It's great that you're open to exploring optio...
6,7de9ac56-4b0f-486c-acb1-63f8df7c4f8d,Save for house deposit,12,it. \n\nWhich of these options sounds most app...,It sounds like you’re considering your options...
7,7de9ac56-4b0f-486c-acb1-63f8df7c4f8d,Save for house deposit,14,I'm really interested in the idea of using ind...,Great choice! Index funds and ETFs can be an e...
8,7de9ac56-4b0f-486c-acb1-63f8df7c4f8d,Save for house deposit,16,: Many brokerages allow you to set up automati...,If you have any specific questions about any o...
9,7de9ac56-4b0f-486c-acb1-63f8df7c4f8d,Save for house deposit,18,I appreciate the guidance! I think I have a so...,I'm glad to hear you feel more confident about...


In [17]:
data.shape[0]

42

The dataset contains five conversations, one per profile.

To facilitate its use for training or fine-tuning tasks, a separate column has been created for each role participating in the conversation. Each row contains a complete interaction, consisting of a question/response pair between the two roles.

The sequence field indicates the position of the interaction within the conversation, which is identified by the `id_conversation` field.